In [8]:
%%time

from pymongo import MongoClient
from pprint import pprint
import json

client = MongoClient('mongodb://localhost:27017')
db = client.spire
status = db.command('serverStatus')
runs = db.runs

match = {
    'event.character_chosen': 'IRONCLAD', 
    'event.floor_reached': {
        '$gte': 40
    }, 
    'event.ascension_level': {
        '$gte': 15
    },
    "event.relics": {
        '$nin': ['PrismaticShard']
    },
    "event.is_beta": False,
    "event.is_endless": False,
}

result = runs.aggregate([
    {
        "$match": match,
    },
    {
        "$project": {
            "event.master_deck": 1,
        },
    }, {
        "$unwind": {
            "path": "$event.master_deck",
        },
    }, {
        "$group": {
            "_id": "$event.master_deck",
        },
    },
]);
cards = [item['_id'] for item in result]

result = runs.aggregate([
    {
        "$match": match,
    },
    {
        "$project": {
            "event.relics": 1,
        },
    }, {
        "$unwind": {
            "path": "$event.relics",
        },
    }, {
        "$group": {
            "_id": "$event.relics",
        },
    },
]);
relics = [item['_id'] for item in result]

items = {
    'relics': relics,
    'cards': cards,
}
with open('all_items.json', 'w') as f:
    json.dump(items, f)

CPU times: user 12.2 ms, sys: 500 µs, total: 12.7 ms
Wall time: 937 ms


In [10]:
with open('all_items.json') as f:
    loaded = json.load(f)
loaded['cards'][:5]

['Feel No Pain+1', 'Magnetism+1', 'Trip', 'Evolve+1', 'Enlightenment+1']

In [12]:
'Feel No Pain' in loaded['cards']

True

In [16]:
'Feel No Pain' in set(loaded['cards'])

True

In [15]:
'Burning Blood' in loaded['relics']

True